In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/opt/workspace/{}/notebooks/Clickstream_Analytics/scr'.format(curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser)) 
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib
import json
import subprocess
import tqdm
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

from spark_connector import SparkConnector
from sparkdb_loader import spark
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.functions import udf 
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

import loader as load

# from corpora_process import utils
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [4]:
LOCAL_PATH = './csv'
HDFS_PATH = '/user/{}'.format(curruser)

In [5]:
def load_as_csv(dst_df, file_name, save_local=True):
    local_path = os.path.join(LOCAL_PATH, file_name)
    hdfs_path = os.path.join(HDFS_PATH, file_name)

    dst_df \
        .coalesce(1).write \
        .format('com.databricks.spark.csv') \
        .mode('overwrite') \
        .option('sep', ',') \
        .option('header', 'true') \
        .save(hdfs_path)

    if save_local:
        proc = subprocess.Popen(['hdfs', 'dfs', '-getmerge', hdfs_path, local_path])
        proc.communicate()
        print('Import to file:', local_path)
    else:
        print('Import to HDFS:', hdfs_path)

In [6]:
conn_schema = 'sbx_team_digitcamp'

In [7]:
%%time
sp = spark(schema='sklod_external_google_analytics',
           queue = 'ektov1-av_ca-sbrf-ru',
           process_label="anthony_",
           numofinstances=20, 
           numofcores=8,
           dynamic_alloc=False,
           kerberos_auth=False
          ) 
print(sp.sc.version)
hive = sp.sql

2.4.0.cloudera2
CPU times: user 41.9 ms, sys: 45.2 ms, total: 87.1 ms
Wall time: 24 s


In [9]:
schema = 'sbx_t_team_speech_analytics' #'dl_t_team_speech_analytics'
conn_schema = 'sbx_team_digitcamp'

# Telemarketing

In [10]:
tbl = 'telemarketing'

In [11]:
tele = hive.sql("select * from {}.{}".format(schema,tbl))

In [12]:
tele.count()

3247900

In [33]:
tele.filter("ucid is not Null").count()

270054

In [13]:
tele = tele.withColumn('phone_number', f.substring('dialednumber', -10, 10))
tele = tele.withColumn('call_dt', f.from_unixtime(f.round(f.unix_timestamp('interactiongmtstarttime')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()))
tele.printSchema()
tele.show()

root
 |-- interactionid: long (nullable = true)
 |-- ucid: string (nullable = true)
 |-- interactionlocalstarttime: timestamp (nullable = true)
 |-- interactionlocalstoptime: timestamp (nullable = true)
 |-- interactiongmtstarttime: timestamp (nullable = true)
 |-- interactiongmtstoptime: timestamp (nullable = true)
 |-- interactionduration: long (nullable = true)
 |-- agents: string (nullable = true)
 |-- switches: string (nullable = true)
 |-- phonenumbers: string (nullable = true)
 |-- dialednumber: string (nullable = true)
 |-- channel_0: string (nullable = true)
 |-- channel_1: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- call_dt: timestamp (nullable = true)



,interactionid,ucid,interactionlocalstarttime,interactionlocalstoptime,interactiongmtstarttime,interactiongmtstoptime,interactionduration,agents,switches,phonenumbers,dialednumber,channel_0,channel_1,text,date,phone_number,call_dt
0,6678244567188314919,00501141821554899958,2019-04-10 15:39:41.850,2019-04-10 15:40:19.053,2019-04-10 12:39:41.850,2019-04-10 12:40:19.053,372031320,"54612, 4835","108, 122",989037137788,989037137788,аллё. добры. знаете это телефон я уже наверное...,добрый день. сбербанк клиентский менеджер екат...,Канал 0: аллё\nКанал 1: добрый день\nКанал 0: ...,2019-04-10,9037137788,2019-04-10 12:40:00
1,6747540883829230691,None,2019-10-14 09:24:47.850,2019-10-14 09:25:21.320,2019-10-14 06:24:47.850,2019-10-14 06:25:21.320,334701469,"4675, 44394","108, 122",89198636381,89198636381,аллё. да. некогда вот извините некогда. пока. ...,добрый день. добрый день меня зовут наталья я ...,Канал 0: аллё\nКанал 1: добрый день. добрый де...,2019-10-14,9198636381,2019-10-14 06:25:00
2,6747514482665275034,None,2019-10-14 07:42:20.423,2019-10-14 07:43:05.493,2019-10-14 04:42:20.423,2019-10-14 04:43:05.493,450708862,"4164, 54646","108, 122",89089288067,89089288067,аллё. я вас слушаю. не прервётся да. у меня та...,добрый день. добрый день меня зовут мария я сп...,Канал 1: добрый день\nКанал 0: аллё\nКанал 1: ...,2019-10-14,9089288067,2019-10-14 04:42:00
3,6747652617403449056,None,2019-10-14 16:38:22.183,2019-10-14 16:41:48.493,2019-10-14 13:38:22.183,2019-10-14 13:41:48.493,2063092045,"4197, 54661","108, 122",89205577756,89205577756,здравствуйте. да. да слушаю. нет у меня не пол...,добрый день. меня зовут эльвира я специалист с...,Канал 1: добрый день\nКанал 0: здравствуйте\nК...,2019-10-14,9205577756,2019-10-14 13:38:00
4,6747701348077212009,00501082411571071641,2019-10-14 19:47:32.543,2019-10-14 19:50:42.010,2019-10-14 16:47:32.543,2019-10-14 16:50:42.010,1894672338,54959,"108, 122",989052298522,59019,добрый день семьсот шесть двести сорок. да. ма...,светлана здравствуйте назовите пожалуйста. сем...,Канал 1: светлана здравствуйте назовите пожалу...,2019-10-14,59019,2019-10-14 16:48:00
5,6747629677983119377,None,2019-10-14 15:09:21.723,2019-10-14 15:11:45.947,2019-10-14 12:09:21.723,2019-10-14 12:11:45.947,1442232456,"44335, 3348","108, 122",89109651998,89109651998,здравствуйте. это я только мне уже звонили. вы...,добрый день. меня зовут ольга я специалист сбе...,Канал 1: добрый день\nКанал 0: здравствуйте\nК...,2019-10-14,9109651998,2019-10-14 12:09:00
6,6747520044647907450,None,2019-10-14 08:03:55.427,2019-10-14 08:04:56.520,2019-10-14 05:03:55.427,2019-10-14 05:04:56.520,610929240,"54153, 3994","108, 122",89021745734,89021745734,аллё. здравствуйте. по вопросу. слушаю вас. да...,добрый день. добрый день меня зовут. специалис...,Канал 0: аллё\nКанал 1: добрый день. добрый де...,2019-10-14,9021745734,2019-10-14 05:04:00
7,6747564235566422371,None,2019-10-14 10:55:24.513,2019-10-14 10:55:51.270,2019-10-14 07:55:24.513,2019-10-14 07:55:51.270,267570312,"2340, 52512","108, 122",89617422595,89617422595,нет. нет. угу до_свидания,добрый день меня зовут анастасия я специалист ...,Канал 1: добрый день меня зовут анастасия я сп...,2019-10-14,9617422595,2019-10-14 07:55:00
8,6747515625126575877,None,2019-10-14 07:46:46.383,2019-10-14 07:50:42.743,2019-10-14 04:46:46.383,2019-10-14 04:50:42.743,2363601264,"42117, 4684","108, 122",89231371327,89231371327,кб автоматика. не спрашивают. советской сторон...,добрый день меня зовут наталья являюсь специал...,Канал 1: добрый день меня зовут наталья являюс...,2019-10-14,9231371327,2019-10-14 04:47:00
9,6747627152542349116,None,2019-10-14 14:59:33.813,2019-10-14 15:01:06.950,2019-10-14 11:59:33.813,2019-10-14 12:01:06.950,931369681,"54826, 4382","108, 122",89509623142,89509623142,здравствуйте. да я слушаю вас. да конечно пожа...,меня зовут екатерина я со сбербанком. ваше вни...,Канал 0: здравствуйте\nКанал 1: меня зовут ека...,2019-10-14,9509623142,2019-10-14 12:00:00


In [23]:
# camps inns + iskra phones
sql  = '''SELECT DISTINCT 
                  c.inn as inn
                 ,cast(c.active_flg as FLOAT) as active_flg
                 ,c.rko_open
                 --,to_date(cast(c.rko_open/1000 as TIMESTAMP)) as rko_open
                 ,c.registration_date as reg_dt
                 --,to_date(cast(c.registration_date/1000 as TIMESTAMP)) as reg_dt
                 ,c.tm_camp_last as tm_name
                 --,c.tm_date_last as tm_dt
                 ,cast(c.tm_date_last/1000 as TIMESTAMP) as tm_dt
                 ,c.tm_result_last as tm_result
                 ,a.phone as phone
            FROM sbx_team_digitcamp.SAS_A1_21_TE_COL_N a,
                 sbx_team_digitcamp.sas_y1_tumanov_inn_rko_tm c
            WHERE a.inn = c.inn
                  AND a.phone is not null
      '''
camp = hive.sql(sql)
camp = camp.withColumn('active_flg', f.col('active_flg').cast(IntegerType()))
camp = camp.withColumn('phone', f.substring('phone', -10, 10))
camp.printSchema()
camp.show()

root
 |-- inn: string (nullable = true)
 |-- active_flg: integer (nullable = true)
 |-- rko_open: long (nullable = true)
 |-- reg_dt: long (nullable = true)
 |-- tm_name: string (nullable = true)
 |-- tm_dt: timestamp (nullable = true)
 |-- tm_result: string (nullable = true)
 |-- phone: string (nullable = true)



,inn,active_flg,rko_open,reg_dt,tm_name,tm_dt,tm_result,phone
0,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ223-0219,2019-02-26,С,9282828120
1,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ393-1019,2019-10-25,Н,9282828120
2,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ445-0120,2020-02-05,О,9282828120
3,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ9-0118,2018-03-29,О,9282828120
4,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМРО19-0519,2019-06-10,С,9282828120
5,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМРО23-0719,2019-07-29,О,9282828120
6,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ223-0219,2019-02-26,С,9286628780
7,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ393-1019,2019-10-25,Н,9286628780
8,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ445-0120,2020-02-05,О,9286628780
9,010100083920,1,1339531200000,1072645200000,ЦА-ПРИВЛТМ9-0118,2018-03-29,О,9286628780


In [24]:
# set 10 hours difference between timestamps
days = 1
join_condition = (camp.phone == tele.phone_number) & \
                 (f.abs(f.unix_timestamp(camp.tm_dt) - f.unix_timestamp(tele.call_dt)) <= days*24*60*60)

new_data = camp.join(tele, join_condition)
# new_data = new_data.withColumnRenamed('inn', 'inn2')

# targets = hive.sql('''SELECT inn, fd_month, tm_date, target
#                               FROM {}.{}'''.format(INN_TABLE))

# targets = targets.withColumn('tm_date', f.col('tm_date').cast(t.DateType()))
# targets = targets.withColumn('inn', f.col('inn').cast(t.LongType()))

# cond = (f.to_date(new_data.tm_dt) == f.to_date(targets.tm_date)) & \
#        (new_data.inn2 == targets.inn)

# new_data = new_data.join(targets, cond)

In [25]:
new_data.cache()

DataFrame[inn: string, active_flg: int, rko_open: bigint, reg_dt: bigint, tm_name: string, tm_dt: timestamp, tm_result: string, phone: string, interactionid: bigint, ucid: string, interactionlocalstarttime: timestamp, interactionlocalstoptime: timestamp, interactiongmtstarttime: timestamp, interactiongmtstoptime: timestamp, interactionduration: bigint, agents: string, switches: string, phonenumbers: string, dialednumber: string, channel_0: string, channel_1: string, text: string, date: string, phone_number: string, call_dt: timestamp]

In [31]:
new_data.select("interactionid", "ucid", "phone_number", "inn" ).filter("ucid is not Null").show()

,interactionid,ucid,phone_number,inn
0,6759039516275709107,00501087761573711491,8482478519,6321194334
1,6839973712497812988,00501008521592555466,9139127408,5402052985
2,6785109924812764146,00501131141579781490,9180164924,230113359525
3,6753918695488163457,00501014381572519215,9221501790,6685095040
4,6802496974631287137,00501011501583829738,9639251004,7743892794
5,6772167839730184195,00501060161576768195,8123270000,781100925643
6,6772137749189304771,00501072371576761184,8123270000,781100925643
7,6750195040446906640,00501095771571652246,9277802560,632402550451
8,6830794233514899794,00501150911590418205,9312875916,471258140031
9,6744992555153303720,00501151121570440933,9164124844,771385047844


In [26]:
new_data.select('inn', 'phone').distinct().count()

1372948

In [32]:
new_data.filter("ucid is not Null").select('inn', 'phone').distinct().count()

2724

In [36]:
res = new_data.select("interactionid", "ucid", "phone_number", "inn" )
res.registerTempTable("tmp_inn_cell")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "telemarketing_cell2inn"))
hive.sql("create table {}.{} select * from tmp_inn_cell".format("sbx_team_digitcamp", "telemarketing_cell2inn"))

DataFrame[]

In [26]:
rep = hive.sql("select * from {}.db_sme_report_report_kc_all".format("sbx_internal_kakb_sas"))

In [28]:
rep.take(1)

[Row(organization_id='020100280234_0', inn='020100280234', kpp='0', response_tracking_cd='235290101', crm_id='null', campaign_nm='ЦА-ПРИВЛТМ104-0818', processed_dttm='2018-08-15 16:01:24.0', call_date='2018-08-16 00:00:00.0', call_time='2018-08-16 12:53:20.0', duration=None, oper_pin='null', oper_fio='null', set_date='1960-01-01 05:56:51.0', call_result='Недозвон', call_result_ext='null', recall_date='null', recall_time='null', if_out='Не выбыл', n_of_tries=2.0, org_name='ИП ХАРРАСОВ РЕНАТ МИРЗОВИЧ', aoh='null', call_id='-20180816-125320', result_last='1', id_campaign='ЦА-ПРИВЛТМ104-0818', comm='null', tb='Уральский банк', gosb='Башкирское головное отделение № 8598', vsp='null', visit_dt='null', lpr_fio='null', lpr_occ='null', lpr_tel='null', lpr_email='null', product_dir='1-115HHZC', product_type='null', product_new='null', aggr_type='null', aggr_deg='null', add1_bank_nm='null', add1_bank='null', add2_bank_nm='null', add2_bank='null', add3_bank_nm='null', add3_bank='null', add4_bank_n

In [29]:
rep.select(f.max(f.to_timestamp("CALL_DATE"))).show()

,max(to_timestamp(`CALL_DATE`))
0,2021-02-16


In [30]:
rep.select("CALL_RESULT", "CALL_RESULT_EXT", "COMM", "CRM_ID", "INN").show()

,CALL_RESULT,CALL_RESULT_EXT,COMM,CRM_ID,CALL_ID
0,Недозвон,null,null,null,-20180816-125320
1,Недозвон,null,null,null,-20180816-085158
2,Недозвон,null,неуд,1-QJ6S7UZ,-20180816-100647
3,Недозвон,null,null,1-QJ4NVT1,-20180816-085939
4,Недозвон,Абонент не в сети/Выключен,null,null,9068723066-20180817-115207
5,Перенос,Клиент не берет трубку,null,1-RDF2FBK,89106768110-20180917-134741
6,Перенос,Системный перенос,null,1-RDEQ9TT,89142716495-20180914-073640
7,Перенос,Системный перенос,null,1-RDEQ9TT,89142716495-20180914-081359
8,Перенос,Клиент не берет трубку,null,1-RDEQ9TT,89142716495-20180914-084239
9,Перенос,Клиент не берет трубку,null,1-RDEQ9TT,89142716495-20180914-091912


# AIST_INTERACTION_HISTORY

In [10]:
aist = hive.sql("select * from {}.AIST_INTERACTION_HISTORY".format(conn_schema))

In [11]:
aist.select(f.max("INTERACTION_START_DATE").alias('max_interac')).show()

,max_interac
0,2021-01-17 20:51:37


In [24]:
aist.filter("ACTION_ITEM_TASK_CRM_ID is not Null").count()

27474788

In [10]:
aist.dtypes

[('INTERACTION_ID', 'string'),
 ('INTERACTION_LOGIN', 'string'),
 ('INTERACTION_START_DATE', 'timestamp'),
 ('INTERACTION_END_DATE', 'timestamp'),
 ('INTERACTION_COMMENTARY', 'string'),
 ('INTERACTION_RESULT', 'string'),
 ('INTERACTION_REASON', 'string'),
 ('INTERACTION_RESCHEDULE_DATE', 'string'),
 ('INTERACTION_ROLE_NAME', 'string'),
 ('INTERACTION_CHANNEL', 'string'),
 ('CORPORATE_INTERACTION_ORG_ID', 'string'),
 ('CALL_ID', 'string'),
 ('CALL_START_DATE', 'timestamp'),
 ('CALL_END_DATE', 'timestamp'),
 ('CALL_INCOMING_CALL', 'int'),
 ('CALL_RESULT', 'string'),
 ('CALL_EMPLOYEE_ID', 'string'),
 ('CALL_PHONE_NUMBER', 'string'),
 ('CALL_UCID', 'string'),
 ('CALL_AGENT_LOGIN', 'string'),
 ('CALL_CALL_TYPE', 'string'),
 ('ACTION_ITEM_RESULT_ID', 'string'),
 ('ACTION_ITEM_RESULT_RESULT', 'string'),
 ('ACTION_ITEM_RESULT_COMMENTARY', 'string'),
 ('ACTION_ITEM_RESULT_REASON', 'string'),
 ('ACTION_ITEM_RESULT_RESCHEDULE_DA', 'string'),
 ('ACTION_ITEM_RESULT_PRODUCT_NAME', 'string'),
 ('ACTI

In [11]:
aist = aist.withColumn("COALESCE_INTERACTION_COMMENTARY", f.when((f.col("INTERACTION_COMMENTARY")=='null')&
                                                                 (f.col("ACTION_ITEM_RESULT_COMMENTARY")!='null'),
                                                                 f.col("ACTION_ITEM_RESULT_COMMENTARY")
                                                                ).otherwise(f.col("INTERACTION_COMMENTARY")))\
           .withColumn("COALESCE_INTERACTION_REASON", f.when((f.col("INTERACTION_REASON")=='null')&
                                                             (f.col("ACTION_ITEM_RESULT_REASON")!='null'),
                                                                f.col("ACTION_ITEM_RESULT_REASON")
                                                            ).otherwise(f.col("INTERACTION_REASON")))

In [18]:
aist.filter("INTERACTION_REASON =='null' and ACTION_ITEM_RESULT_REASON != 'null'")\
    .select("INTERACTION_REASON",
            "ACTION_ITEM_RESULT_REASON").show(50)

,INTERACTION_REASON,ACTION_ITEM_RESULT_REASON
0,null,Телефон не отвечает/ занят
1,null,Телефон не отвечает/ занят
2,null,Номер не активен (заблокирован или не использу...
3,null,Продукт не подходит
4,null,Нет потребности в настоящее время
5,null,Подтвержден интерес к продукту
6,null,Не удалось связаться с клиентом
7,null,Нет потребности в настоящее время
8,null,Не удалось связаться с клиентом
9,null,Не удалось связаться с клиентом


In [20]:
aist.filter("INTERACTION_COMMENTARY == 'null' and ACTION_ITEM_RESULT_COMMENTARY != 'null'")\
    .select("INTERACTION_COMMENTARY",
            "ACTION_ITEM_RESULT_COMMENTARY",
            "COALESCE_INTERACTION_COMMENTARY",
            "COALESCE_INTERACTION_REASON").show(50)

,INTERACTION_COMMENTARY,ACTION_ITEM_RESULT_COMMENTARY,COALESCE_INTERACTION_COMMENTARY,COALESCE_INTERACTION_REASON
0,null,не участвует в тендерах,не участвует в тендерах,Продукт не подходит
1,null,Месячный фонд оплаты труда: 20000 рублей; Коли...,Месячный фонд оплаты труда: 20000 рублей; Коли...,Подтвержден интерес к продукту
2,null,"перезвонить в марте месяце , так как возможно ...","перезвонить в марте месяце , так как возможно ...",Телефон не отвечает/ занят
3,null,Сумма - 560 тыс. / Ставка - 17 % / Срок финанс...,Сумма - 560 тыс. / Ставка - 17 % / Срок финанс...,Подтвержден интерес к продукту
4,null,не нужно,не нужно,Категорический отказ
5,null,не надо,не надо,Продуктовое предложение не было рассмотрено
6,null,ип1,ип1,Продукт не подходит
7,null,"себе не выплачивает, штата нет","себе не выплачивает, штата нет",Продукт не подходит
8,null,"ЦКР Ставрополь, счет зарезервирован, ПУ Легкий...","ЦКР Ставрополь, счет зарезервирован, ПУ Легкий...",Подтвержден интерес к продукту
9,null,Сейчас БК не актуальна/ Возможно рассмотрит пр...,Сейчас БК не актуальна/ Возможно рассмотрит пр...,Нет потребности в настоящее время


In [40]:
comments = aist.select("INTERACTION_ID", "COALESCE_INTERACTION_COMMENTARY").distinct().collect()

In [50]:
reasons = aist.select("INTERACTION_ID", "COALESCE_INTERACTION_REASON").distinct().collect()

In [51]:
len(comments), len(reasons)

(21953186, 20895189)

## Export comments

In [38]:
with open('./csv/COALESCE_INTERACTION_COMMENTARY', 'w+') as fout:
    for item in comments:
        if item['COALESCE_INTERACTION_COMMENTARY'] != None:
            item = item['COALESCE_INTERACTION_COMMENTARY']+'\n'
            fout.writelines(item)

In [48]:
sdf = aist.filter("COALESCE_INTERACTION_COMMENTARY != 'null'").select("INTERACTION_ID", "COALESCE_INTERACTION_COMMENTARY")
load_as_csv(sdf,'km_comments.csv')

Import to file: ./csv/km_comments.csv


### Check DateTime for the First non Null CALL_UCID

In [70]:
isucid_min_dttm = aist.filter('CALL_UCID is not Null')\
                      .select(f.min('INTERACTION_START_DATE').alias('isucid_min_dttm')).collect()[0]['isucid_min_dttm']

In [31]:
isucid_min_dttm = isucid_min_dttm.split('.')[0]
# isucid_min_dttm = re.sub(r'\d{2}\:\d{2}\:\d{2}','00:00:00',isucid_min_dttm)
isucid_min_dttm

'2019-12-08 21:04:00'

## Filter and Rebuild AIST Table

In [12]:
dct = \
dict(list(zip(['ACTION_ITEM_ORG_ID',
               'CORPORATE_INTERACTION_ORG_ID',
               'ACTION_ITEM_TASK_CRM_ID',
               'ACTION_ITEM_PROD_OFFER_CRM_ID'], ['']*4)))

In [13]:
aist = aist.fillna(dct)
aist = aist.replace(to_replace = {'null': ''}, subset = list(dct.keys())+['CALL_PHONE_NUMBER'])

In [20]:
refdatetime = datetime.fromtimestamp(datetime.timestamp(datetime.now())-40*24*60*60)
refdate = datetime.strftime(refdatetime, format='%Y-%m-%d')

In [21]:
aist_filter = \
aist.select('INTERACTION_ID',
            'CALL_ID',
            'CALL_UCID',
            #'CALL_EMPLOYEE_ID',
            #'CALL_AGENT_LOGIN',
            #'INTERACTION_LOGIN',
            #'ACTION_ITEM_LOGIN',
            #f.from_unixtime(f.round(f.unix_timestamp('CALL_START_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('CALL_START_DATE'),
            #f.from_unixtime(f.round(f.unix_timestamp('CALL_END_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('CALL_END_DATE'),
            f.from_unixtime(f.round(f.unix_timestamp('INTERACTION_START_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('INTERACTION_START_DATE'),
            f.from_unixtime(f.round(f.unix_timestamp('INTERACTION_END_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('INTERACTION_END_DATE'),           
            f.from_unixtime(f.round(f.unix_timestamp('ACTION_ITEM_CREATE_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('ACTION_ITEM_CREATE_DATE'),
            #f.from_unixtime(f.round(f.unix_timestamp('ACTION_ITEM_FACT_START_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('ACTION_ITEM_FACT_START_DATE'),
            #f.from_unixtime(f.round(f.unix_timestamp('ACTION_ITEM_FACT_END_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('ACTION_ITEM_FACT_END_DATE'),            
            'CALL_INCOMING_CALL',
            f.substring('CALL_PHONE_NUMBER',-10,10).alias('CALL_PHONE_NUMBER'),
            f.split(f.col('ACTION_ITEM_ORG_ID'),'_').getItem(0).alias('ACTION_ITEM_ORG_ID'),
            #f.split(f.col('CORPORATE_INTERACTION_ORG_ID'),'_').getItem(0).alias('CORPORATE_INTERACTION_ORG_ID'),
            f.split(f.col('ACTION_ITEM_TASK_CRM_ID'),'_').getItem(0).alias('ACTION_ITEM_TASK_CRM_ID'),
            f.split(f.col('ACTION_ITEM_PROD_OFFER_CRM_ID'),'_').getItem(0).alias('ACTION_ITEM_PROD_OFFER_CRM_ID'),
            'ACTION_ITEM_RESULT_PRODUCT_NAME',
            #'ACTION_ITEM_ID_PROD_OFFER_LITE',
            'ACTION_ITEM_PRODUCT_ID',
            'ACTION_ITEM_PRODUCT_DESCRIPTION',
            'COALESCE_INTERACTION_COMMENTARY',
            'COALESCE_INTERACTION_REASON'
            ).filter((f.col('CALL_INCOMING_CALL')==0)&
                     (f.col('CALL_PHONE_NUMBER')!='')&
                     (f.col('INTERACTION_START_DATE').cast(TimestampType()) >= f.to_timestamp(f.lit(refdate))) #f.lit(isucid_min_dttm).cast(TimestampType())
                    )

In [22]:
aist_filter.cache()

DataFrame[INTERACTION_ID: string, CALL_ID: string, CALL_UCID: string, INTERACTION_START_DATE: timestamp, INTERACTION_END_DATE: timestamp, ACTION_ITEM_CREATE_DATE: timestamp, CALL_INCOMING_CALL: int, CALL_PHONE_NUMBER: string, ACTION_ITEM_ORG_ID: string, ACTION_ITEM_TASK_CRM_ID: string, ACTION_ITEM_PROD_OFFER_CRM_ID: string, ACTION_ITEM_RESULT_PRODUCT_NAME: string, ACTION_ITEM_PRODUCT_ID: string, ACTION_ITEM_PRODUCT_DESCRIPTION: string, COALESCE_INTERACTION_COMMENTARY: string, COALESCE_INTERACTION_REASON: string]

In [23]:
aist_filter.count()

0

In [21]:
aist_filter.filter("ACTION_ITEM_RESULT_PRODUCT_NAME is not Null").count()

12217371

In [61]:
aist_filter.columns

['INTERACTION_ID',
 'CALL_ID',
 'CALL_UCID',
 'INTERACTION_START_DATE',
 'INTERACTION_END_DATE',
 'ACTION_ITEM_CREATE_DATE',
 'CALL_INCOMING_CALL',
 'CALL_PHONE_NUMBER',
 'ACTION_ITEM_ORG_ID',
 'CORPORATE_INTERACTION_ORG_ID',
 'ACTION_ITEM_TASK_CRM_ID',
 'ACTION_ITEM_PROD_OFFER_CRM_ID',
 'ACTION_ITEM_RESULT_PRODUCT_NAME',
 'ACTION_ITEM_ID_PROD_OFFER_LITE',
 'ACTION_ITEM_PRODUCT_ID',
 'ACTION_ITEM_PRODUCT_DESCRIPTION',
 'COALESCE_INTERACTION_COMMENTARY',
 'COALESCE_INTERACTION_REASON']

In [62]:
aist_filter.filter("ACTION_ITEM_TASK_CRM_ID != '' ").select( 'ACTION_ITEM_ORG_ID',
                                                             'CORPORATE_INTERACTION_ORG_ID',
                                                             'ACTION_ITEM_TASK_CRM_ID',
                                                             'ACTION_ITEM_PROD_OFFER_CRM_ID',
                                                             'ACTION_ITEM_RESULT_PRODUCT_NAME',
                                                             'ACTION_ITEM_ID_PROD_OFFER_LITE',
                                                             'ACTION_ITEM_PRODUCT_ID',
                                                             'ACTION_ITEM_PRODUCT_DESCRIPTION',
                                                             'COALESCE_INTERACTION_COMMENTARY').show()

,ACTION_ITEM_ORG_ID,CORPORATE_INTERACTION_ORG_ID,ACTION_ITEM_TASK_CRM_ID,ACTION_ITEM_PROD_OFFER_CRM_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,ACTION_ITEM_ID_PROD_OFFER_LITE,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_PRODUCT_DESCRIPTION,COALESCE_INTERACTION_COMMENTARY
0,1-726LVHJ,1-726LVHJ,1-2NKFFA7X,,Бизнес-карта,null,1-115HHYO,ЦКР.Онбординг «Бизнес-карта». Сообщить клиенту...,875 не звонить зв главбуху
1,1-2MVSQH23,1-2MVSQH23,1-2NCIQECY,,Открытие банковского счета,null,54809,ЦКР.Клиент открыл р/счет 24.09.2020. Канал отк...,null
2,1-11FT2TKX,1-11FT2TKX,1-2M6S20D0,,Зарплатные проекты,null,368989657566,ЦКР.First call. ПК. (А) Клиент не выплачивал з...,null
3,AIST50f31839,,AIST6b7aa037,,null,null,AISTNotProfile,#ЮЛ,10 001 передан в УСО
4,1-18KJHZX,1-18KJHZX,1-2P4CWYDV,,Зарплатные проекты,null,368989657566,ЦКР.First call. ПК. ОТ9 Клиент снизил на 71% в...,null
5,1-18KJHZX,1-18KJHZX,1-2P4CWYDV,,Зарплатные проекты,null,368989657566,ЦКР.First call. ПК. ОТ9 Клиент снизил на 71% в...,null
6,1-18KJHZX,1-18KJHZX,1-2P4CWYDV,,Зарплатные проекты,null,368989657566,ЦКР.First call. ПК. ОТ9 Клиент снизил на 71% в...,null
7,1-21KXB6Y,1-21KXB6Y,1-2OZTB4S7,,Зарплатные проекты,null,368989657566,ЦКР.Онбординг «Зарплатный проект». 08.09.2020 ...,null
8,1-F2LP25H,1-F2LP25H,1-2OZSDUBO,,Бизнес-карта,null,1-115HHYO,ЦКР.Онбординг «Бизнес-карта». Сообщить клиенту...,null
9,1-F2LP25H,1-F2LP25H,1-2OZSDUBO,,Бизнес-карта,null,1-115HHYO,ЦКР.Онбординг «Бизнес-карта». Сообщить клиенту...,null


In [71]:
aist_filter.filter("ACTION_ITEM_PROD_OFFER_CRM_ID is not Null").select("ACTION_ITEM_PROD_OFFER_CRM_ID", 
                                                                       "ACTION_ITEM_ORG_ID",
                                                                       "ACTION_ITEM_TASK_CRM_ID"
                                                                      ).show()

,ACTION_ITEM_PROD_OFFER_CRM_ID,ACTION_ITEM_ORG_ID,ACTION_ITEM_TASK_CRM_ID
0,1-2M6YSX93,1-27VRU0LT,
1,1-2NG27CN8,1-17727T5,
2,1-2M74RHJ4,1-UQPX9HT,
3,1-2M74RHJ6,1-UQPX9HT,
4,1-2LRN7CZH,1-TWAUV1T,
5,,1-726LVHJ,1-2NKFFA7X
6,1-2NKJ1SSJ,1-2G4NOAZ2,
7,1-2NKELCFY,1-2G4NOAZ2,
8,1-2N71CQOL,1-NCVP1X4,
9,1-2NM23U9R,1-2LK31ZU5,


## MA_TASK

In [25]:
matsk = hive.sql("select * from {}.MA_TASK".format(conn_schema))

In [44]:
matsk.show()

,HOST_TASK_ID,CREATE_DT,END_PLAN_DT,END_FACT_DT,TASK_INFO_TXT,TASK_TYPE_VAL,CREATOR_ID,EXECUTOR_FIO,EXECUTOR_ID,INN,KPP,TASK_RESULT_TYPE_NAME,RESULT_CMT_TXT,HOST_APPL_ID,HOST_LEAD_ID,STATUS_NAME,CAMPAIGN_NM,CRTR_EMP_ID,HOST_CUST_ORG_ID,CRM_CAMPAIGN_ID
0,1-2FU4U8GW,2020-07-12,2020-07-27,2020-07-28,ЗВОНОК КЛИЕНТУ ИНН 500603567650. ЗАДАЧА ДЛЯ КМ...,Звонок,1-2ZTNFQW,Личенко Юлия Александровна,1-155AGQW8,500603567650,None,Другое (в комментарии),КМ ЦКР совершил звонок,None,1-2FU47VDL,Закрыта,TRIPPLE_ENTITY_MMB,None,1-MF1TMK4,1-2BUD51ZJ
1,1-2OUU6VB9,2020-10-09,2020-10-17,2020-10-16,ЦКР.КЛИЕНТ ОТКРЫЛ Р/СЧЕТ 08.10.2020. КАНАЛ ОТК...,Знакомство с КМ ЦКР,1-2ZTNFQW,Козлов Алексей Анатольевич,1-1H5DL8C6,544052563459,None,Выполнено,Подтвержден интерес к продукту; Организационна...,None,None,Закрыта,ЦА-ОНБОРДИНГММБПИЛОТ-0620,None,1-2KAAR2N6,1-2CQ134UK
2,1-2RMTR9NF,2020-10-29,2020-11-05,NaT,ЦКР.КЛИЕНТ ОТКРЫЛ Р/СЧЕТ 28.10.2020. КАНАЛ ОТК...,Знакомство с КМ ЦКР,1-2ZTNFQW,Козлов Алексей Анатольевич,1-1H5DL8C6,5505065619,550501001,Назначен повторный звонок,АИСТ: Клиент думает;Телефон не отвечает/ занят...,None,1-2RMT240H,В работе,ЦА-ОНБОРДИНГММБПИЛОТ-0620,None,1-2Q53ZFLD,1-2CQ134UK
3,1-2N5KHPII,2020-09-25,2020-09-29,2020-09-29,ВСТРЕЧА С КЛИЕНТОМ ИНН 229000057844. ЗАДАЧА ДЛ...,Переговоры/встреча,1-2ZTNFQW,Шведова Екатерина Андреевна,1-V2QGSSW,229000057844,None,Сервисная задача отработана,None,None,1-2N5JT8YO,Закрыта,TRIPPLE_ENTITY_MMB_MKK,None,1-EYLB1C,1-2EQEXKMQ
4,1-2NQCR6F8,2020-09-30,2020-10-07,NaT,ЦКР. ТЕХНИЧЕСКИЙ ЗВОНОК ЗАПЛАНИРОВАННЫЙ ДЛЯ КМ...,Звонок,0-1,Весельева Виктория Николаевна,1-L6HMQLA,682700743411,None,None,None,None,1-2M6XMF8P,Отменена,None,None,1-JUA1FTV,None
5,1-2H5E22VM,2020-07-28,2020-08-06,2020-08-05,ЦКР.ДО ОТКРЫТИЯ СЧЕТА КЛИЕНТ ПРОЯВЛЯЛ ИНТЕРЕС ...,FollowUp,1-2ZTNFQW,Козлов Алексей Анатольевич,1-1H5DL8C6,5038154070,503801001,Другое (в комментарии),АИСТ: Не удалось связаться с клиентом (недозвон);,None,None,Закрыта,ЦА-ФАОНБ-0720,None,1-2FPUM04K,1-2GE2L6JG
6,1-2LR9W158,2020-09-11,2020-09-26,2020-09-15,ЗВОНОК КЛИЕНТУ ИНН 310501027537. ЗАДАЧА ДЛЯ КМ...,Звонок,1-2ZTNFQW,Весельева Виктория Николаевна,1-L6HMQLA,310501027537,None,Другое (в комментарии),КМ ЦКР совершил звонок,None,1-2LR8B5MZ,Закрыта,TRIPPLE_ENTITY_MMB,None,1-1TB6OK95,1-2BUD51ZJ
7,1-2LQVFRGP,2020-09-11,2020-09-26,2020-09-25,ЗВОНОК КЛИЕНТУ ИНН 401700464498. ЗАДАЧА ДЛЯ КМ...,Звонок,1-2ZTNFQW,Личенко Юлия Александровна,1-155AGQW8,401700464498,None,Другое (в комментарии),КМ ЦКР совершил звонок,None,1-2LQT9L9F,Закрыта,TRIPPLE_ENTITY_MMB,None,1-XZ350S,1-2BUD51ZJ
8,1-2OQNY065,2020-10-08,2020-10-23,2020-10-13,"ЦКР.ПК. В АВГУСТЕ ДОЛЯ ОПЕРАЦИЙ, ПРОВОДИМЫХ ПО...",Расширение сотрудничества,1-2ZTNFQW,Весельева Виктория Николаевна,1-L6HMQLA,614109729480,None,Другое (в комментарии),Другое (в комментарии); Только ТЭ от СБ. ;,None,None,Закрыта,ЦА-ПОТЕНЦИАЛММБЦКР-0719,None,1-1C2J0BT3,1-2CXO08NG
9,1-2OQSNZR9,2020-10-08,2020-10-24,NaT,ЗВОНОК КЛИЕНТУ ИНН 362000123002. ЗАДАЧА ДЛЯ КМ...,Звонок,1-2ZTNFQW,Весельева Виктория Николаевна,1-L6HMQLA,362000123002,None,None,None,None,1-2OQSIMJD,Отменена,TRIPPLE_ENTITY_MMB,None,1-3OAO898,1-2BUD51ZJ


In [64]:
matsk_filter = matsk.filter("CREATE_DT>=timestamp('{}')".format(refdate)).select('CREATE_DT', 
                                                                                 'END_FACT_DT',
                                                                                 'HOST_CUST_ORG_ID',
                                                                                 'INN')

In [65]:
matsk_filter.cache()

DataFrame[CREATE_DT: timestamp, END_FACT_DT: timestamp, HOST_CUST_ORG_ID: string, INN: string]

In [66]:
matsk_filter.count()

19942206

### Check Intersection between MA_TASK and AIST by ORD_ID

In [73]:
matsk_filter.filter('HOST_CUST_ORG_ID = "1-TWAUV1T"').show()

,CREATE_DT,END_FACT_DT,HOST_CUST_ORG_ID,INN
0,2020-09-29,2020-09-30,1-TWAUV1T,440120609778
1,2020-09-23,2020-09-29,1-TWAUV1T,440120609778
2,2020-10-06,2020-10-09,1-TWAUV1T,440120609778
3,2020-09-30,2020-09-30,1-TWAUV1T,440120609778
4,2020-09-29,2020-09-29,1-TWAUV1T,440120609778
5,2020-09-30,2020-10-05,1-TWAUV1T,440120609778
6,2020-10-15,NaT,1-TWAUV1T,440120609778
7,2020-09-30,2020-09-30,1-TWAUV1T,440120609778
8,2020-10-06,2020-10-06,1-TWAUV1T,440120609778
9,2020-10-06,2020-10-09,1-TWAUV1T,440120609778


In [72]:
aist_filter.filter('ACTION_ITEM_ORG_ID = "1-TWAUV1T"').show()

,INTERACTION_ID,CALL_ID,CALL_UCID,INTERACTION_START_DATE,INTERACTION_END_DATE,ACTION_ITEM_CREATE_DATE,CALL_INCOMING_CALL,CALL_PHONE_NUMBER,ACTION_ITEM_ORG_ID,ACTION_ITEM_TASK_CRM_ID,ACTION_ITEM_PROD_OFFER_CRM_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_PRODUCT_DESCRIPTION,COALESCE_INTERACTION_COMMENTARY,COALESCE_INTERACTION_REASON
0,d97bf13c-0f3c-4f0a-b4fe-9cf5880ad184,97c93e32-d3ab-4b6c-8659-1d525fe847e3,501104301601374642,2020-09-29 13:17:00,2020-09-29 13:23:00,2020-09-11 17:54:00,0,9066092084,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,null,Не дозвонились до клиента
1,8df4489c-9638-4c3a-927b-619922b19f56,aab8b330-1ab3-4923-85ad-d76210e2d7d6,501131191600869461,2020-09-23 16:57:00,2020-09-23 17:00:00,2020-09-11 17:54:00,0,9109570903,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,уточнит,Телефон не отвечает/ занят
2,8df4489c-9638-4c3a-927b-619922b19f56,5495ac4c-6046-4d40-9cc0-f634e1b843a3,501119681600869409,2020-09-23 16:57:00,2020-09-23 17:00:00,2020-09-11 17:54:00,0,9108008008,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,уточнит,Телефон не отвечает/ занят
3,ad107df1-1a18-4671-9911-e6a63562aa59,9353e06d-c8b4-4476-90e3-c030e2cd7590,501024911601461084,2020-09-30 13:18:00,2020-09-30 13:20:00,2020-09-11 17:54:00,0,9108008008,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,Владимир бухгалтер,Телефон не отвечает/ занят
4,ad107df1-1a18-4671-9911-e6a63562aa59,4ecbee43-a3ea-44ec-b338-3b844016e851,501042151601461136,2020-09-30 13:18:00,2020-09-30 13:20:00,2020-09-11 17:54:00,0,9109570903,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,Владимир бухгалтер,Телефон не отвечает/ занят
5,7927d5a9-6f10-4308-8e39-af681807587a,e84f0980-a79c-49f5-b475-4b5ddf1bebc4,501054921603363992,2020-10-22 13:53:00,2020-10-22 13:56:00,2020-09-11 17:54:00,0,9066092084,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,высокий процент,Обслуживается в другом банке
6,7927d5a9-6f10-4308-8e39-af681807587a,ec21d53f-383a-4d58-82ac-8c20d9f4bac5,501065521603364027,2020-10-22 13:53:00,2020-10-22 13:56:00,2020-09-11 17:54:00,0,9108008008,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,высокий процент,Обслуживается в другом банке
7,7927d5a9-6f10-4308-8e39-af681807587a,3f380d98-1bf4-41b6-ab49-8585cf5bb59b,501079351603364076,2020-10-22 13:53:00,2020-10-22 13:56:00,2020-09-11 17:54:00,0,9109570903,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,высокий процент,Обслуживается в другом банке
8,4ed065af-d288-4a94-8946-8113fa2ec403,695e1ba6-f94b-49f3-9904-59a0535d7cd5,501115111601453047,2020-09-30 11:04:00,2020-09-30 11:07:00,2020-09-11 17:54:00,0,9108008008,1-TWAUV1T,,1-2LRN7CZH,Эквайринг,162546926598,First call. ПК.Клиент регулярно вносит наличны...,null,Не дозвонились до клиента
9,2f091bc5-4bf3-45f9-a864-ff6959886346,cafb05d5-06ac-4999-b97b-cd3fc648cac0,501123831602750047,2020-10-15 11:21:00,2020-10-15 11:30:00,2020-09-30 14:29:00,0,9108008008,1-TWAUV1T,,1-2NQEDBWA,Оплата по QR коду,1-164W2L7O,ЦКР.Кросс-продажа,инт;,null


## Join Filtered AIST with MA_TASK

In [169]:
# # set 2 hours difference between timestamps
# days = 1
# join_condition = ( (f.abs(f.unix_timestamp(aist_filter.CALL_START_DATE) - 
#                           f.unix_timestamp(matsk_filter.CREATE_DT))) <= days*24*60*60)|\
#                  (aist_filter.CALL_START_DATE >= matsk_filter.CREATE_DT)&\
#                  (aist_filter.CALL_START_DATE <= matsk_filter.END_FACT_DT)

# aist_filter_task = aist_filter.join(matsk_filter, on=join_condition, how='left_outer')

In [77]:
days = 2
join_condition =  (
                      (f.abs(
                               f.unix_timestamp(aist_filter.INTERACTION_START_DATE) - 
                               f.unix_timestamp(matsk_filter.CREATE_DT)
                            ) <= days*24*60*60
                      )|\
                      ((aist_filter.INTERACTION_START_DATE >= matsk_filter.CREATE_DT)&\
                       (aist_filter.INTERACTION_START_DATE <= matsk_filter.END_FACT_DT)
                      )
                  )&\
                  (aist_filter.ACTION_ITEM_ORG_ID == matsk_filter.HOST_CUST_ORG_ID)

aist_filter_task = aist_filter.join(matsk_filter, on=join_condition, how='left_outer')

## Export Joined Table

In [39]:
import subprocess

In [81]:
aist_filter_task.registerTempTable("tmp_aist_task")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "aist_hist_matask_join"))
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("aist_hist_matask_join")])

hive.sql("create table {}.{} select * from tmp_aist_task".format("sbx_team_digitcamp", "aist_hist_matask_join"))

DataFrame[]

## Check aist_hist_matask_join

In [78]:
aistmatask = hive.sql('''select 
                         ACTION_ITEM_ORG_ID, 
                         ACTION_ITEM_PRODUCT_ID,
                         ACTION_ITEM_RESULT_PRODUCT_NAME,
                         COALESCE_INTERACTION_COMMENTARY,
                         CALL_UCID,
                         CALL_PHONE_NUMBER,
                         INTERACTION_START_DATE,
                         INTERACTION_END_DATE,
                         INN
                         from sbx_team_digitcamp.aist_hist_matask_join''')

In [83]:
aistmatask.filter("INN is not Null").count()

19674689

In [76]:
aistmatask.filter("CALL_UCID = '501073771594718344'").show(20)

,ACTION_ITEM_ORG_ID,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,COALESCE_INTERACTION_COMMENTARY,CALL_UCID,CALL_PHONE_NUMBER,INTERACTION_START_DATE,INTERACTION_END_DATE,INN
0,1-2FAW7JPH,54809,ПУ Легкий старт,null,501073771594718344,9872080001,2020-07-14 12:19:00,2020-07-14 12:20:00,None


In [53]:
aistmatask.select(f.min("CALL_START_DATE")).show()

,min(CALL_START_DATE)
0,2020-01-06 03:48:00


In [59]:
aistmatask.filter((f.col("CALL_START_DATE") > pd.to_datetime('2020-01-14')) &
                  (f.col("CALL_START_DATE") < pd.to_datetime('2020-01-15')) &
                  (f.col("CALL_PHONE_NUMBER") == '9246656035')).show()

,ACTION_ITEM_ORG_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,CALL_UCID,CALL_PHONE_NUMBER,CALL_START_DATE,CALL_END_DATE,INN


## Apply User Defined RegExp on comments field

In [114]:
aistmatask = \
aistmatask.withColumn("isNeedForCall",
                        f.regexp_extract(f.col('COALESCE_INTERACTION_COMMENTARY'),
                                         '''(.*подумает.*|.*перезвон.*ближе.*|.*не\s*удобно.*договорились.*|
                                             .*перезвон.*|.*клиент просил.*|.*просил.*(?:набрать)*|.*позвони.*|.*(?!<не) интерес.*)''',1)
                     )

In [107]:
aistmatask.filter("isNeedForCall !='' and INN is not Null").show(70)

,ACTION_ITEM_ORG_ID,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,COALESCE_INTERACTION_COMMENTARY,CALL_UCID,CALL_PHONE_NUMBER,INTERACTION_START_DATE,INTERACTION_END_DATE,INN,isNeedForCall
0,1-10J2BTMJ,1-415WQ1A,Защита сотрудников,подумает;,501036661587126940,9200199726,2020-04-17 15:35:00,2020-04-17 15:42:00,520202343166,подумает;
1,1-10N7C6TV,368989657566,Зарплатные проекты,"нет сотрудников и счета. Обсудили ЛС- дорого, ...",501123631595494872,9274013295,2020-07-23 12:00:00,2020-07-23 12:09:00,165060646092,"нет сотрудников и счета. Обсудили ЛС- дорого, ..."
2,1-110EHOEA,368989657566,Зарплатные проекты,собирается в собственность приобретать кассу э...,501091301593761632,9376405240,2020-07-03 10:34:00,2020-07-03 10:37:00,634003542484,собирается в собственность приобретать кассу э...
3,1-110EHOEA,368989657566,Зарплатные проекты,попросил через 1 час набрать,501111981599111007,9376405240,2020-09-03 08:30:00,2020-09-03 08:31:00,634003542484,попросил через 1 час набрать
4,1-110EHOEA,368989657566,Зарплатные проекты,попросил через 1 час набрать,501111981599111007,9376405240,2020-09-03 08:30:00,2020-09-03 08:31:00,634003542484,попросил через 1 час набрать
...,...,...,...,...,...,...,...,...,...,...
65,1-13JRITK6,368989657566,Зарплатные проекты,позвонить бухгалтеру ;,501081031592480707,9020061385,2020-06-18 14:45:00,2020-06-18 14:49:00,732817190840,позвонить бухгалтеру ;
66,1-13JRITK6,368989657566,Зарплатные проекты,позвонить бухгалтеру ;,501081031592480707,9020061385,2020-06-18 14:45:00,2020-06-18 14:49:00,732817190840,позвонить бухгалтеру ;
67,1-13YPDMUF,368989657566,Зарплатные проекты,подумает;,501088551589335203,9143612927,2020-05-13 05:00:00,2020-05-13 05:05:00,753609876111,подумает;
68,1-13YPDMUF,368989657566,Зарплатные проекты,подумает;,501088551589335203,9143612927,2020-05-13 05:00:00,2020-05-13 05:05:00,753609876111,подумает;


In [116]:
df = aistmatask.filter("isNeedForCall !='' and INN is not Null").toPandas()

In [121]:
df.drop(columns=["CALL_UCID", "CALL_PHONE_NUMBER"], inplace=True)

In [122]:
df.head()

,ACTION_ITEM_ORG_ID,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,COALESCE_INTERACTION_COMMENTARY,INTERACTION_START_DATE,INTERACTION_END_DATE,INN
0,1-10J2BTMJ,1-415WQ1A,Защита сотрудников,подумает;,2020-04-17 15:35:00,2020-04-17 15:42:00,520202343166
1,1-10N7C6TV,368989657566,Зарплатные проекты,"нет сотрудников и счета. Обсудили ЛС- дорого, ...",2020-07-23 12:00:00,2020-07-23 12:09:00,165060646092
2,1-110EHOEA,368989657566,Зарплатные проекты,собирается в собственность приобретать кассу э...,2020-07-03 10:34:00,2020-07-03 10:37:00,634003542484
3,1-110EHOEA,368989657566,Зарплатные проекты,попросил через 1 час набрать,2020-09-03 08:30:00,2020-09-03 08:31:00,634003542484
4,1-110EHOEA,368989657566,Зарплатные проекты,попросил через 1 час набрать,2020-09-03 08:30:00,2020-09-03 08:31:00,634003542484


In [123]:
df.to_excel("./csv/aist_ckr_needforcall.xlsx", index=False, encoding='utf-8')

## Analyse results from NICE datamart: cell_inn_interaction_end_date

In [57]:
topics = pd.read_excel('./csv/Nice_Topics.xlsx',header=None)

In [58]:
topics.reset_index(inplace=True, level=0)
topics.rename(columns={'index':'topic_num',0:'topics'}, inplace=True)
topics.head()

,topic_num,topics
0,0,негатив
1,1,общая лексика
2,2,отсутствие интереса
3,3,сведения об операциях р/с
4,4,будущий интерес


In [59]:
topics_sdf = hive.createDataFrame(topics, schema=StructType([StructField('topic_num', IntegerType()), 
                                                      StructField('topics', StringType())
                                                     ])
                          )

In [60]:
dialogs = hive.sql('''select inn inn_tm, 
                             substring(ucid,-18,18) ucid, \
                             phone_number, date, topic_num from sbx_team_digitcamp.cell_inn_interaction_end_date''')

In [61]:
dialogs = dialogs.join(topics_sdf, on='topic_num', how='left_outer')

In [62]:
dialogs.cache()

DataFrame[topic_num: int, inn_tm: string, ucid: string, phone_number: string, date: timestamp, topics: string]

In [63]:
dialogs.filter("ucid is not Null").show()

,topic_num,inn_tm,ucid,phone_number,date,topics
0,3,231105877420,501066251581517247,8612109810,2020-02-12,сведения об операциях р/с
1,3,850200938920,501137441590470839,9642729497,2020-05-26,сведения об операциях р/с
2,23,782500001084,501008281598873140,9119244344,2020-08-31,зарплатный проект
3,69,422370200499,501128831581312825,9134158705,2020-02-10,получение наличных/банковские продукты
4,69,5047219446,501152941582189765,9160182933,2020-02-20,получение наличных/банковские продукты
5,69,351700153942,501072221598432457,9218326347,2020-08-26,получение наличных/банковские продукты
6,69,552805476761,501150911591616198,9277609833,2020-06-08,получение наличных/банковские продукты
7,69,9731026642,501075961579866115,9263583030,2020-01-24,получение наличных/банковские продукты
8,69,7820068520,501072861584099150,9657769635,2020-03-13,получение наличных/банковские продукты
9,69,505304166554,501103711595855782,9779573121,2020-07-27,получение наличных/банковские продукты


In [67]:
dialogs.filter("inn_tm is not Null").count()

84712

In [75]:
# dialogs = dialogs.select(f.col("inn_tm").alias("inn_tm_"), 
#                          f.col("phone_number").alias("phone_number_"), 
#                          f.col("topics").alias("topics_"))

## Join with AISTMATASK

In [32]:
days = 2
join_condition =  (
                     ((f.abs(
                               f.unix_timestamp(aistmatask.INTERACTION_START_DATE) - 
                               f.unix_timestamp(dialogs.date)
                            ) <= days*24*60*60
                      )|\
                      ((dialogs.date >= aistmatask.INTERACTION_START_DATE)&\
                       (dialogs.date <= aistmatask.INTERACTION_END_DATE)
                      )
                     )&(dialogs.phone_number == aistmatask.CALL_PHONE_NUMBER) 
                  )|\
                  (dialogs.ucid == aistmatask.CALL_UCID)
    
dialogsaist = dialogs.join(aistmatask, on=join_condition, how='left_outer')

In [33]:
dialogsaist.cache()

DataFrame[topic_num: int, inn_tm: string, ucid: string, phone_number: string, date: timestamp, topics: string, ACTION_ITEM_ORG_ID: string, ACTION_ITEM_PRODUCT_ID: string, ACTION_ITEM_RESULT_PRODUCT_NAME: string, CALL_UCID: string, CALL_PHONE_NUMBER: string, INTERACTION_START_DATE: timestamp, INTERACTION_END_DATE: timestamp, INN: string]

In [35]:
cols = \
[
 'INN' ,
 'inn_tm',
 'ucid',
 'phone_number',
 'date',
 'topics',
 'ACTION_ITEM_ORG_ID',
 'ACTION_ITEM_PRODUCT_ID',
 'ACTION_ITEM_RESULT_PRODUCT_NAME',
 'INTERACTION_START_DATE',
 'INTERACTION_END_DATE'
]

In [38]:
dialogsaist = dialogsaist.select(*[cols])
dialogsaist = dialogsaist.select([f.col(col).alias(col.upper()) for col in dialogsaist.columns])

In [40]:
dialogsaist.registerTempTable("tmp_aist_dialogs")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "telemarketing_aist_cell2inn"))
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("telemarketing_aist_cell2inn")])

hive.sql("create table {}.{} select * from tmp_aist_dialogs".format("sbx_team_digitcamp", "telemarketing_aist_cell2inn"))

DataFrame[]

## telemarketing_aist_cell2inn

In [68]:
dialogsaist = hive.sql("select * from {}.telemarketing_aist_cell2inn".format(conn_schema))

In [69]:
dialogsaist = \
dialogsaist.select(*[f.col(col).alias(col.split("upper(")[-1][:-1].upper()) for col in dialogsaist.columns])

In [55]:
dialogsaist.show()

,INN,INN_TM,UCID,PHONE_NUMBER,DATE,TOPICS,ACTION_ITEM_ORG_ID,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,INTERACTION_START_DATE,INTERACTION_END_DATE
0,None,231403463350,None,9181138943,2020-02-19 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-1U53YXVR,54809,ПУ ЛЕГКИЙ СТАРТ,2020-02-18 15:51:00,2020-02-18 15:54:00
1,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 15:21:00,2020-07-14 15:21:00
2,None,1655423746,None,9872080001,2020-07-13 00:00:00,Е-ИНВОЙСИНГ/ДОКУМЕНТООБОРОТ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 15:21:00,2020-07-14 15:21:00
3,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-13 13:02:00,2020-07-13 13:03:00
4,None,1655423746,None,9872080001,2020-07-13 00:00:00,Е-ИНВОЙСИНГ/ДОКУМЕНТООБОРОТ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-13 13:02:00,2020-07-13 13:03:00
5,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 12:19:00,2020-07-14 12:20:00
6,None,1655423746,None,9872080001,2020-07-13 00:00:00,Е-ИНВОЙСИНГ/ДОКУМЕНТООБОРОТ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 12:19:00,2020-07-14 12:20:00
7,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-13 15:09:00,2020-07-13 15:09:00
8,None,1655423746,None,9872080001,2020-07-13 00:00:00,Е-ИНВОЙСИНГ/ДОКУМЕНТООБОРОТ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-13 15:09:00,2020-07-13 15:09:00
9,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 09:15:00,2020-07-14 09:16:00


In [70]:
dialogsaist.filter("(INN is not Null) or (INN_TM is not Null)").count()

86443

In [72]:
df = dialogsaist.toPandas()

In [73]:
df.head()

,INN,INN_TM,UCID,PHONE_NUMBER,DATE,TOPICS,ACTION_ITEM_ORG_ID,ACTION_ITEM_PRODUCT_ID,ACTION_ITEM_RESULT_PRODUCT_NAME,INTERACTION_START_DATE,INTERACTION_END_DATE
0,None,231403463350,None,9181138943,2020-02-19 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-1U53YXVR,54809,ПУ ЛЕГКИЙ СТАРТ,2020-02-18 15:51:00,2020-02-18 15:54:00
1,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 15:21:00,2020-07-14 15:21:00
2,None,1655423746,None,9872080001,2020-07-13 00:00:00,Е-ИНВОЙСИНГ/ДОКУМЕНТООБОРОТ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-14 15:21:00,2020-07-14 15:21:00
3,None,1655423746,None,9872080001,2020-07-13 00:00:00,ПОЛУЧЕНИЕ НАЛИЧНЫХ/БАНКОВСКИЕ ПРОДУКТЫ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-13 13:02:00,2020-07-13 13:03:00
4,None,1655423746,None,9872080001,2020-07-13 00:00:00,Е-ИНВОЙСИНГ/ДОКУМЕНТООБОРОТ,1-2FAW7JPH,54809,ПУ ЛЕГКИЙ СТАРТ,2020-07-13 13:02:00,2020-07-13 13:03:00


In [74]:
df.to_excel("./csv/inn_telem_topics.xlsx", index=False, encoding='utf-8')

## CALLS_UUP_DETAIL

In [121]:
callsup = hive.sql("select * from {}.CALLS_UUP_DETAIL".format(conn_schema))

In [122]:
callsup.show()

,TAB_,FIO,SERVICE,UCID,ROW_TIME,LOGIN,DURATION,INCOMING_CALLS,INCOMING_CALLS_30,INCOMING_CALLS_60,OUTCOMING_CALLS,OUTCOMING_CALLS_30,OUTCOMING_CALLS_60,ROW_DATE,ROW_DATETIME
0,1702731,Кульпина Ксения Михайловна,УУП,501084161580879591,813,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 11:13:11
1,1702731,Кульпина Ксения Михайловна,УУП,501150971580899365,1342,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 16:42:45
2,1702731,Кульпина Ксения Михайловна,УУП,501007661580899429,1343,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 16:43:49
3,1702731,Кульпина Ксения Михайловна,УУП,501003941580899407,1343,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 16:43:27
4,1702731,Кульпина Ксения Михайловна,УУП,501126461580888964,1049,54045,222,0,0,0,1,1,1,2020-02-05 03:00:00,2020-02-05 13:49:24
5,1702731,Кульпина Ксения Михайловна,УУП,501055411580879268,807,54045,32,0,0,0,1,1,0,2020-02-05 03:00:00,2020-02-05 11:07:48
6,1702731,Кульпина Ксения Михайловна,УУП,501057141580125855,1450,54045,0,0,0,0,1,0,0,2020-01-27 03:00:00,2020-01-27 17:50:55
7,1702731,Кульпина Ксения Михайловна,УУП,501068781580113878,1131,54045,353,0,0,0,1,1,1,2020-01-27 03:00:00,2020-01-27 14:31:18
8,1702731,Кульпина Ксения Михайловна,УУП,501055571580113141,1119,54045,0,0,0,0,1,0,0,2020-01-27 03:00:00,2020-01-27 14:19:01
9,1702731,Кульпина Ксения Михайловна,УУП,501136541580110557,1035,54045,0,0,0,0,1,0,0,2020-01-27 03:00:00,2020-01-27 13:35:57


# Select records from nice table

In [147]:
nice = hive.sql("select * from {}.nice".format(schema))

In [148]:
nice = nice.withColumn('phone_number', f.substring('phonenumbers', -10, 10))
nice = nice.withColumn('call_dt', f.from_unixtime(f.round(f.unix_timestamp('interactiongmtstarttime')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()))
nice.printSchema()

root
 |-- interactionid: long (nullable = true)
 |-- ucid: string (nullable = true)
 |-- interactionlocalstarttime: timestamp (nullable = true)
 |-- interactionlocalstoptime: timestamp (nullable = true)
 |-- interactiongmtstarttime: timestamp (nullable = true)
 |-- interactiongmtstoptime: timestamp (nullable = true)
 |-- interactionduration: long (nullable = true)
 |-- agents: string (nullable = true)
 |-- switches: string (nullable = true)
 |-- phonenumbers: string (nullable = true)
 |-- dialednumber: string (nullable = true)
 |-- channel_0: string (nullable = true)
 |-- channel_1: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- call_dt: timestamp (nullable = true)



In [152]:
nice = nice.replace({'NULL':''},['ucid'])

In [154]:
nice.show()

,interactionid,ucid,interactionlocalstarttime,interactionlocalstoptime,interactiongmtstarttime,interactiongmtstoptime,interactionduration,agents,switches,phonenumbers,dialednumber,channel_0,channel_1,text,date,phone_number,call_dt
0,6644299885779690925,,2019-01-09 04:16:59.720,2019-01-09 04:22:03.663,2019-01-09 01:16:59.720,2019-01-09 01:22:03.663,3039440000,57222,108,,55191,добрый день елена подскажите пожалуйста мы хот...,сбербанк елена здравствуйте. как могу обращать...,Канал 1: сбербанк елена здравствуйте\nКанал 0:...,2019-01-09,,2019-01-09 01:17:00
1,6644305434967610385,00501134431546997910,2019-01-09 04:38:33.523,2019-01-09 04:43:30.480,2019-01-09 01:38:33.523,2019-01-09 01:43:30.480,2969567247,54399,108,9084476178,59033,здравствуйте александр у меня счёт у вас это к...,меня зовут александр чем могу вам помочь. а ск...,Канал 1: меня зовут александр чем могу вам пом...,2019-01-09,9084476178,2019-01-09 01:39:00
2,6644309300438178118,00501011661546998810,2019-01-09 04:53:33.220,2019-01-09 04:54:50.173,2019-01-09 01:53:33.220,2019-01-09 01:54:50.173,769528079,54761,108,,59033,меня зовут мария чем могу вам помочь. при подп...,мария здравствуйте меня зовут василий операцио...,Канал 0: меня зовут мария чем могу вам помочь\...,2019-01-09,,2019-01-09 01:54:00
3,6644316301234872455,00501033021547000179,2019-01-09 05:20:45.890,2019-01-09 05:21:55.157,2019-01-09 02:20:45.890,2019-01-09 02:21:55.157,692672308,54226,108,869143129,59005,да у меня с карты я не могу не рассчитаться де...,меня зовут лариса здравствуйте чем могу вам по...,Канал 1: меня зовут лариса здравствуйте чем мо...,2019-01-09,869143129,2019-01-09 02:21:00
4,6644320643446810069,00501049951547001336,2019-01-09 05:37:38.197,2019-01-09 05:44:20.607,2019-01-09 02:37:38.197,2019-01-09 02:44:20.607,4024107628,57463,108,74996731111,59029,оксана добрый день подскажите пожалуйста вот у...,сбербанк оксана здравствуйте. скажите пожалуйс...,Канал 1: сбербанк оксана здравствуйте\nКанал 0...,2019-01-09,4996731111,2019-01-09 02:38:00
5,6644320961184213395,,2019-01-09 05:38:46.467,2019-01-09 05:39:31.927,2019-01-09 02:38:46.467,2019-01-09 02:39:31.927,454630000,52003,108,,51570,аллё. да да. а мне вечером пожалуйста позвонит...,добрый день ирина васильевна. меня зовут алёна...,Канал 0: аллё\nКанал 1: добрый день ирина васи...,2019-01-09,,2019-01-09 02:39:00
6,6644321553879215948,,2019-01-09 05:41:03.957,2019-01-09 05:41:20.980,2019-01-09 02:41:03.957,2019-01-09 02:41:20.980,170240000,54509,108,,53630,абонента временно недоступен. пожалуйста позво...,None,Канал 0: абонента временно недоступен. пожалуй...,2019-01-09,,2019-01-09 02:41:00
7,6644353302378130151,00501115601547009054,2019-01-09 07:44:17.793,2019-01-09 07:53:56.627,2019-01-09 04:44:17.793,2019-01-09 04:53:56.627,5788341770,54272,108,989137506671,59003,добрый день не могу отправить не могут не може...,елена здравствуйте чем я могу вам помочь. скаж...,Канал 1: елена здравствуйте чем я могу вам пом...,2019-01-09,9137506671,2019-01-09 04:44:00
8,6644354255860870632,00501124141547009240,2019-01-09 07:48:01.077,2019-01-09 07:48:46.843,2019-01-09 04:48:01.077,2019-01-09 04:48:46.843,457660132,52929,108,,59005,здравствуйте подскажите пожалуйста заходим рас...,здравствуйте меня зовут алёна чем могу вам пом...,Канал 1: здравствуйте меня зовут алёна чем мог...,2019-01-09,,2019-01-09 04:48:00
9,6644356850021119934,00501003301547009858,2019-01-09 07:58:08.787,2019-01-09 07:58:27.723,2019-01-09 04:58:08.787,2019-01-09 04:58:27.723,189356742,57981,108,869255744,57960,None,да. сбербанке ксения здравствуйте. я вас слуша...,Канал 1: да. сбербанке ксения здравствуйте. я ...,2019-01-09,869255744,2019-01-09 04:58:00


In [157]:
# nice_aist = nice.filter("ucid=''")
nice_aist = nice.filter("call_dt<timestamp('{}')".format(isucid_min_dttm))
nice_aist.count()

7571708

In [158]:
nice_aist.select(f.max('call_dt')).show()

,max(call_dt)
0,2019-12-08 21:02:00


In [ ]:
# set 2 hours difference between timestamps
days = 1
join_condition = ( (f.abs(f.unix_timestamp(nice_aist.call_dt) - 
                          f.unix_timestamp(uup_pp_part.create_date))) <= days*24*60*60)

new_data = reswithaist_part.join(uup_pp_part, join_condition)

## Join by USID with AIST records

In [16]:
aist_filter.count()

1815486

In [19]:
join_condition = (enter.ucid == aist_filter.call_ucid)
reswithaist = enter.join(aist_filter, join_condition)

In [18]:
reswithaist.count()

1022532

### Select data from additional iskra tables

In [22]:
calls_upp = hive.sql("select * from {}.CALLS_UUP_DETAIL".format(schema))
uup_pp = hive.sql("select * from {}.FV_CKR_UUP_PP".format(schema))
uup_val = hive.sql("select * from {}.FV_CKR_UUP_VAL".format(schema))

In [23]:
calls_upp = calls_upp.select([f.col(col).alias(col.lower()) for col in calls_upp.columns])
calls_upp = calls_upp.withColumn('tab', f.col('tab_').cast(IntegerType()))
calls_upp = calls_upp.drop('tab_')
calls_upp.printSchema()

root
 |-- fio: string (nullable = true)
 |-- service: string (nullable = true)
 |-- ucid: long (nullable = true)
 |-- row_time: long (nullable = true)
 |-- login: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- incoming_calls: long (nullable = true)
 |-- incoming_calls_30: long (nullable = true)
 |-- incoming_calls_60: long (nullable = true)
 |-- outcoming_calls: long (nullable = true)
 |-- outcoming_calls_30: long (nullable = true)
 |-- outcoming_calls_60: long (nullable = true)
 |-- row_date: timestamp (nullable = true)
 |-- row_datetime: timestamp (nullable = true)
 |-- tab: integer (nullable = true)



In [24]:
uup_pp = uup_pp.select([f.col(col).alias(col.lower()) for col in uup_pp.columns])
uup_pp = uup_pp.withColumn('tab', f.col('tab_num').cast(IntegerType()))
uup_pp = uup_pp.drop('tab_num')
uup_pp.printSchema()

root
 |-- pp_id: string (nullable = true)
 |-- main_user_id: string (nullable = true)
 |-- inic_user_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- pp_stage: string (nullable = true)
 |-- offer_id: string (nullable = true)
 |-- offer_stage: string (nullable = true)
 |-- offer_status: string (nullable = true)
 |-- offer_source: string (nullable = true)
 |-- org_id: string (nullable = true)
 |-- org_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- inn: float (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_group: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- priority: long (nullable = true)
 |-- partner: long (nullable = true)
 |-- offer_reason: string (nullable = true)
 |-- stage_4: long (nullable = true)
 |-- km_id: string (nullable = true)
 |-- km_fio: string (nullable = true)
 |-- km_status: string (nullable = true)
 |-- gosb: string (nullable = true)
 |-- sector_id: float (nullabl

In [43]:
uup_pp.select(f.col('inn').cast(IntegerType()).cast(StringType()),
              'offer_id',
              'org_id',
              'product_id', 
              'product_group', 
              'product_name').show()

,inn,offer_id,org_id,product_id,product_group,product_name
0,2147483647,1-10TPGQER,1-10TPGQD4,1-HLPSUHX,ПУ,ПУ Легкий старт
1,2147483647,1-11KZL18Z,1-11KZL15Y,1-HLPSUHX,ПУ,ПУ Легкий старт
2,2147483647,1-10QWIE1M,1-10QWIDTV,1-HLPSUHX,ПУ,ПУ Легкий старт
3,2147483647,1-11FJIWU6,1-11FJIWOX,1-HLPSUHX,ПУ,ПУ Легкий старт
4,2147483647,1-11J1KXY0,1-9RVJCN8,1-115HHYO,Бизнес-Карта,Бизнес-карта
5,918002752,1-118NQ90O,1-1Z0EIZK,1-1GUC,ТЭ,Эквайринг
6,2147483647,1-11BKLPDG,1-11BJXU1W,1-HLPSUHX,ПУ,ПУ Легкий старт
7,2147483647,1-119KS3I8,1-119KGTLP,1-OCYGMDT,ПУ,Пакет услуг Хорошая выручка
8,2147483647,1-10Y257MY,1-10Y257LB,1-HLPSUHX,ПУ,ПУ Легкий старт
9,2147483647,1-10SYIEPA,1-17CPMHQ,1-HLPSUHX,ПУ,ПУ Легкий старт


### AIST: Find relevant INN base don the join with FV_CKR_UUP_PP

In [26]:
reswithaist.dtypes

[('interactionid', 'bigint'),
 ('ucid', 'string'),
 ('interactionlocalstarttime', 'timestamp'),
 ('interactionlocalstoptime', 'timestamp'),
 ('interactiongmtstarttime', 'timestamp'),
 ('interactiongmtstoptime', 'timestamp'),
 ('interactionduration', 'bigint'),
 ('agents', 'string'),
 ('switches', 'string'),
 ('phonenumbers', 'string'),
 ('dialednumber', 'string'),
 ('channel_0', 'string'),
 ('channel_1', 'string'),
 ('text', 'string'),
 ('date', 'string'),
 ('phone_number', 'string'),
 ('call_dt', 'timestamp'),
 ('call_ucid', 'string'),
 ('action_item_product_id', 'string'),
 ('action_item_result_product_name', 'string'),
 ('action_item_task_crm_id', 'string'),
 ('action_item_prod_offer_crm_id', 'string'),
 ('corporate_interaction_org_id', 'string'),
 ('action_item_org_id', 'string')]

In [27]:
ref_date_down = '2019-08-01 00:00:00'
ref_date_up = '2020-03-17 00:00:00'
reswithaist_part = reswithaist.filter('''call_dt >= timestamp("{down}") and 
                                         call_dt <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                                up=ref_date_up))
uup_pp_part = uup_pp.filter('''create_date >= timestamp("{down}") and 
                               create_date <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                          up=ref_date_up))

In [32]:
# set 2 hours difference between timestamps
days = 1
join_condition = (reswithaist_part.action_item_product_id == uup_pp_part.product_id)&\
                 ( (f.abs(f.unix_timestamp(reswithaist_part.call_dt) - 
                          f.unix_timestamp(uup_pp_part.create_date))) <= days*2*60*60)

new_data = reswithaist_part.join(uup_pp_part, join_condition)

In [33]:
new_data.drop_duplicates()
new_data.count()

9265404

In [37]:
new_data.show()

,interactionid,ucid,interactionlocalstarttime,interactionlocalstoptime,interactiongmtstarttime,interactiongmtstoptime,interactionduration,agents,switches,phonenumbers,...,rkm_fio,vsp_id,tb_km_id,tb_km_fio,sale_chanel,create_date,create_date_ext,pp_stage_date,stage_4_date,tab
0,6785479541108316084,00501026461579867567,2020-01-24 15:06:09.913,2020-01-24 15:06:39.150,2020-01-24 12:06:09.913,2020-01-24 12:06:39.150,292372461,54035,108,988123477068,...,Уварова Ирина Владимировна,9038/10,1-5FE7-39,Дятлова Екатерина Сергеевна,VKM,2020-01-24 10:26:36,2020-01-24 03:00:00,2020-01-24 03:00:00,2020-01-24 03:00:00,1766286
1,6787365190667145425,00501078531580306597,2020-01-29 17:03:26.317,2020-01-29 17:04:30.507,2020-01-29 14:03:26.317,2020-01-29 14:04:30.507,641897037,44720,108,989119265568,...,Ворфоломеева Ольга Александровна,0017/1,1-8IN8-68,Старостин Виктор Владиславович,VKM,2020-01-29 15:21:52,2020-01-29 03:00:00,2020-01-29 03:00:00,2020-01-29 03:00:00,1724353
2,6785122835484448990,00501017861579784516,2020-01-23 16:01:57.647,2020-01-23 16:02:31.380,2020-01-23 13:01:57.647,2020-01-23 13:02:31.380,337335966,54518,108,988123350616,...,Мироненко Дмитрий Сергеевич,8602/1,1-RT5PG4W,Хроленко Мария Васильевна,VKM,2020-01-23 13:10:14,2020-01-23 03:00:00,2020-01-23 03:00:00,2020-01-23 03:00:00,1679644
3,6786969203272395381,00501047471580214389,2020-01-28 15:26:48.053,2020-01-28 15:27:40.437,2020-01-28 12:26:48.053,2020-01-28 12:27:40.437,523835079,44384,108,989643436601,...,Дорохова Ирина Александровна,9038/43,1-OKXDMYW,Рауткина Юлия Евгеньевна,VKM,2020-01-28 11:17:16,2020-01-28 03:00:00,2020-01-28 03:00:00,2020-01-28 03:00:00,1646835
4,6789580504733661381,00501048051580822386,2020-02-04 16:19:59.863,2020-02-04 16:20:02.130,2020-02-04 13:19:59.863,2020-02-04 13:20:02.130,22666652,54934,108,,...,Едемская Екатерина Игоревна,8637/7,1-G0Y1-7,Реука Татьяна Сергеевна,VKM,2020-02-04 13:49:05,2020-02-04 03:00:00,2020-02-04 03:00:00,2020-02-04 03:00:00,1043297
5,6789580504733661381,00501048051580822386,2020-02-04 16:19:59.863,2020-02-04 16:20:02.130,2020-02-04 13:19:59.863,2020-02-04 13:20:02.130,22666652,54934,108,,...,Головина Елена Юрьевна,8623/9,1-G1C1-4,Лычагина Ольга Вячеславовна,VKM,2020-02-04 14:58:12,2020-02-04 03:00:00,2020-02-04 03:00:00,2020-02-04 03:00:00,1609650
6,6786962361389483960,00501062771580212796,2020-01-28 15:00:15.783,2020-01-28 15:01:32.193,2020-01-28 12:00:15.783,2020-01-28 12:01:32.193,764098717,54538,108,989216525812,...,Дорохова Ирина Александровна,9038/43,1-OKXDMYW,Рауткина Юлия Евгеньевна,VKM,2020-01-28 11:17:16,2020-01-28 03:00:00,2020-01-28 03:00:00,2020-01-28 03:00:00,1646835
7,6787006195825721215,00501083581580223003,2020-01-28 17:50:21.323,2020-01-28 17:50:55.013,2020-01-28 14:50:21.323,2020-01-28 14:50:55.013,336896569,44384,108,989119693467,...,Носачев Игорь Викторович,8621/5,1-8HBL-24,Осипян Екатерина Виталиевна,VKM,2020-01-28 16:11:40,2020-01-28 03:00:00,2020-01-28 03:00:00,2020-01-28 03:00:00,1601015
8,6785451791324614546,00501093671579861073,2020-01-24 13:18:28.680,2020-01-24 13:18:40.090,2020-01-24 10:18:28.680,2020-01-24 10:18:40.090,114098744,44232,108,989219352685,...,Уварова Ирина Владимировна,9038/10,1-5FE7-39,Дятлова Екатерина Сергеевна,VKM,2020-01-24 10:26:36,2020-01-24 03:00:00,2020-01-24 03:00:00,2020-01-24 03:00:00,1766286
9,6790296686235289530,00501038731580989146,2020-02-06 14:39:08.020,2020-02-06 14:39:10.643,2020-02-06 11:39:08.020,2020-02-06 11:39:10.643,26232346,54035,108,,...,Сизова-Касаткина Анастасия Александровна,8619/9,1-13ZKB42O,Паршакова Юлия Николаевна,VKM,2020-02-06 10:24:17,2020-02-06 03:00:00,2020-02-06 03:00:00,2020-02-06 03:00:00,1651837


In [40]:
new_data.filter('ucid = "00501038731580989146"').select('inn',
                                                        'product_id',
                                                        'action_item_product_id',
                                                        'product_name',
                                                        'action_item_result_product_name',
                                                        'call_dt',
                                                        'create_date'
                                                        ).show(30)

,inn,product_id,action_item_product_id,product_name,action_item_result_product_name,call_dt,create_date
0,2.336111e+11,1-1HY8DSLR,1-1HY8DSLR,Партнерская программа – Деловая среда,Партнерская программа,2020-02-06 11:39:00,2020-02-06 10:24:17


### Total match based on the CALLS_UUP_DETAIL datamart

In [60]:
ref_date_down = '2019-08-01 00:00:00'
ref_date_up = '2020-03-17 00:00:00'
calls_upp = calls_upp.filter('''row_datetime >= timestamp("{down}") and 
                                row_datetime <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                            up=ref_date_up))
uup_pp = uup_pp.filter('''create_date >= timestamp("{down}") and 
                          create_date <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                            up=ref_date_up))

In [68]:
calls_upp.count(), uup_pp.count()

(5688837, 1076423)

In [69]:
calls_upp.show()

,fio,service,ucid,row_time,login,duration,incoming_calls,incoming_calls_30,incoming_calls_60,outcoming_calls,outcoming_calls_30,outcoming_calls_60,row_date,row_datetime,tab
0,Кульпина Ксения Михайловна,УУП,501084161580879591,813,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 11:13:11,1702731
1,Кульпина Ксения Михайловна,УУП,501150971580899365,1342,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 16:42:45,1702731
2,Кульпина Ксения Михайловна,УУП,501007661580899429,1343,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 16:43:49,1702731
3,Кульпина Ксения Михайловна,УУП,501003941580899407,1343,54045,0,0,0,0,1,0,0,2020-02-05 03:00:00,2020-02-05 16:43:27,1702731
4,Кульпина Ксения Михайловна,УУП,501126461580888964,1049,54045,222,0,0,0,1,1,1,2020-02-05 03:00:00,2020-02-05 13:49:24,1702731
5,Кульпина Ксения Михайловна,УУП,501055411580879268,807,54045,32,0,0,0,1,1,0,2020-02-05 03:00:00,2020-02-05 11:07:48,1702731
6,Кульпина Ксения Михайловна,УУП,501057141580125855,1450,54045,0,0,0,0,1,0,0,2020-01-27 03:00:00,2020-01-27 17:50:55,1702731
7,Кульпина Ксения Михайловна,УУП,501068781580113878,1131,54045,353,0,0,0,1,1,1,2020-01-27 03:00:00,2020-01-27 14:31:18,1702731
8,Кульпина Ксения Михайловна,УУП,501055571580113141,1119,54045,0,0,0,0,1,0,0,2020-01-27 03:00:00,2020-01-27 14:19:01,1702731
9,Кульпина Ксения Михайловна,УУП,501136541580110557,1035,54045,0,0,0,0,1,0,0,2020-01-27 03:00:00,2020-01-27 13:35:57,1702731


In [80]:
# set 2 hours difference between timestamps
days = 1
join_condition = (calls_upp.tab == uup_pp.tab)&\
                 ( (f.abs(f.unix_timestamp(calls_upp.row_datetime) - 
                          f.unix_timestamp(uup_pp.create_date))) <= days*2*60*60)

new_data = calls_upp.join(uup_pp, join_condition)

In [81]:
new_data = new_data.drop_duplicates()
new_data.cache()

DataFrame[fio: string, service: string, ucid: bigint, row_time: bigint, login: bigint, duration: bigint, incoming_calls: bigint, incoming_calls_30: bigint, incoming_calls_60: bigint, outcoming_calls: bigint, outcoming_calls_30: bigint, outcoming_calls_60: bigint, row_date: timestamp, row_datetime: timestamp, tab: int, dd: string, pp_id: string, main_user_id: string, inic_user_id: string, user_id: string, pp_stage: string, offer_id: string, offer_stage: string, offer_status: string, offer_source: string, org_id: string, org_name: string, segment: string, inn: float, product_id: string, product_group: string, product_name: string, priority: bigint, partner: bigint, offer_reason: string, stage_4: bigint, km_id: string, km_fio: string, km_status: string, gosb: string, sector_id: float, sector_name: string, sector_head: string, group_id: float, group_name: string, group_head: string, chanel: string, tb_id: float, tb_name: string, gosb_id: float, gosb_name: string, rm_id: string, rm_fio: str

In [82]:
new_data.count()
#105208897

17040960

### Join with nice

In [83]:
# days = 3
# join_condition = (f.round(f.abs(f.unix_timestamp(enter.call_dt) - f.unix_timestamp(new_data.CREATE_DATE))/60) <= days*24*60)
join_condition = (enter.ucid == new_data.ucid)
res = enter.join(new_data, join_condition)

In [84]:
res = res.drop_duplicates()
res.count()

7694297

In [86]:
res.show()

,interactionid,ucid,interactionlocalstarttime,interactionlocalstoptime,interactiongmtstarttime,interactiongmtstoptime,interactionduration,agents,switches,phonenumbers,...,rkm_fio,vsp_id,tb_km_id,tb_km_fio,sale_chanel,create_date,create_date_ext,pp_stage_date,stage_4_date,tab
0,6765006870822327265,00501000051575100887,2019-11-30 11:01:44.377,2019-11-30 11:06:44.493,2019-11-30 08:01:44.377,2019-11-30 08:06:44.493,3001159127,57411,108,989057447554,...,Пивоваров Антон Юрьевич,8619/12,1-1CBLNIMA,Думброва Татьяна Вячеславовна,ВСП,2019-11-30 14:57:29,2019-11-30 03:00:00,2019-11-30 03:00:00,2019-11-30 03:00:00,1564357
1,6765006870822327265,00501000051575100887,2019-11-30 11:01:44.377,2019-11-30 11:06:44.493,2019-11-30 08:01:44.377,2019-11-30 08:06:44.493,3001159127,57411,108,989057447554,...,Мамонтов Виктор Андреевич,9038/53,1-19PQMDRK,Уколов Андрей Сергеевич,ВСП,2019-11-30 12:01:47,2019-11-30 03:00:00,2019-11-30 03:00:00,2019-11-30 03:00:00,1564357
2,6765006870822327265,00501000051575100887,2019-11-30 11:01:44.377,2019-11-30 11:06:44.493,2019-11-30 08:01:44.377,2019-11-30 08:06:44.493,3001159127,57411,108,989057447554,...,Мамонтов Виктор Андреевич,9038/53,1-19PQMDRK,Уколов Андрей Сергеевич,ВСП,2019-11-30 12:21:22,2019-11-30 03:00:00,2019-11-30 03:00:00,2019-11-30 03:00:00,1564357
3,6765006870822327265,00501000051575100887,2019-11-30 11:01:44.377,2019-11-30 11:06:44.493,2019-11-30 08:01:44.377,2019-11-30 08:06:44.493,3001159127,57411,108,989057447554,...,Пивоваров Антон Юрьевич,8619/12,1-1CBLNIMA,Думброва Татьяна Вячеславовна,ВСП,2019-11-30 14:57:28,2019-11-30 03:00:00,2019-11-30 03:00:00,2019-11-30 03:00:00,1564357
4,6765006870822327265,00501000051575100887,2019-11-30 11:01:44.377,2019-11-30 11:06:44.493,2019-11-30 08:01:44.377,2019-11-30 08:06:44.493,3001159127,57411,108,989057447554,...,Мамонтов Виктор Андреевич,9038/53,1-19PQMDRK,Уколов Андрей Сергеевич,ВСП,2019-11-30 12:21:23,2019-11-30 03:00:00,2019-11-30 03:00:00,2019-11-30 03:00:00,1564357
5,6765006870822327265,00501000051575100887,2019-11-30 11:01:44.377,2019-11-30 11:06:44.493,2019-11-30 08:01:44.377,2019-11-30 08:06:44.493,3001159127,57411,108,989057447554,...,Пивоваров Антон Юрьевич,8619/12,1-1CBLNIMA,Думброва Татьяна Вячеславовна,ВСП,2019-11-30 14:57:27,2019-11-30 03:00:00,2019-11-30 03:00:00,2019-11-30 03:00:00,1564357
6,6746546100656741739,00501000081570802670,2019-10-11 17:04:39.750,2019-10-11 17:05:29.207,2019-10-11 14:04:39.750,2019-10-11 14:05:29.207,494574873,58919,108,989313922219,...,None,None,None,None,ВСП,2019-10-11 18:43:04,2019-10-11 03:00:00,2019-10-11 03:00:00,2019-10-11 03:00:00,1740182
7,6746546100656741739,00501000081570802670,2019-10-11 17:04:39.750,2019-10-11 17:05:29.207,2019-10-11 14:04:39.750,2019-10-11 14:05:29.207,494574873,58919,108,989313922219,...,None,None,None,None,ВСП,2019-10-11 18:42:59,2019-10-11 03:00:00,2019-10-11 03:00:00,2019-10-11 03:00:00,1740182
8,6746546100656741739,00501000081570802670,2019-10-11 17:04:39.750,2019-10-11 17:05:29.207,2019-10-11 14:04:39.750,2019-10-11 14:05:29.207,494574873,58919,108,989313922219,...,None,None,None,None,ВСП,2019-10-11 18:43:02,2019-10-11 03:00:00,2019-10-11 03:00:00,2019-10-11 03:00:00,1740182
9,6746546100656741739,00501000081570802670,2019-10-11 17:04:39.750,2019-10-11 17:05:29.207,2019-10-11 14:04:39.750,2019-10-11 14:05:29.207,494574873,58919,108,989313922219,...,None,None,None,None,ВСП,2019-10-11 18:43:00,2019-10-11 03:00:00,2019-10-11 03:00:00,2019-10-11 03:00:00,1740182
